In [18]:
import os
import numpy as np
from IPython.display import HTML
from itertools import groupby
import re
from Meter_Rhyme_HMM import unsupervised_HMM
import Meter_Rhyme_HMM_helper

In [19]:
shakespeare = open("data/shakespeare.txt", 'r')

poems = shakespeare.readlines()
split_at = "\n"
final_poems = [list(g)[1:] for k, g in groupby(poems, lambda x: x != split_at) if k]
print("Initial number of poems: {}".format(len(final_poems)))
poem_lengths = [len(poem) for poem in final_poems] 
bad_poems = np.where(np.array(poem_lengths)!= 14)[0]
print ("Sonnets {} and {} are not 14 lines long so we remove them from our list.".format(bad_poems[0], bad_poems[1]))

final_poems = [final_poems[i] for i in np.delete(np.arange(len(final_poems)), bad_poems)]
print("Final number of poems: {}".format(len(final_poems)))
final_poems = [''.join([line.strip(' ') for line in poem]) for poem in final_poems]

Initial number of poems: 154
Sonnets 98 and 125 are not 14 lines long so we remove them from our list.
Final number of poems: 152


In [20]:
# token_map maps words to numbers
# tokenized_poems replaces the words in poems with their corresponding number
tokenized_poems, token_map = Meter_Rhyme_HMM_helper.parse_observations(final_poems)
token_map_r = Meter_Rhyme_HMM_helper.obs_map_reverser(token_map)
tokenized_poems

[[[0, 1, 2, 3, 4, 5],
  [6, 7, 8, 9, 10, 11, 12],
  [13, 14, 15, 16, 17, 18, 19, 20],
  [21, 22, 23, 10, 24, 21, 25],
  [13, 26, 27, 28, 29, 30, 31, 32],
  [33, 34, 35, 36, 37, 38, 39],
  [40, 41, 42, 43, 44, 45],
  [34, 46, 34, 47, 28, 34, 48, 46, 49, 50],
  [26, 6, 51, 52, 15, 53, 54, 55],
  [56, 57, 58, 28, 15, 59, 60],
  [61, 29, 30, 62, 63, 34, 64],
  [56, 22, 65, 66, 67, 68, 69],
  [70, 15, 71, 72, 73, 74, 75, 76],
  [28, 77, 15, 53, 78, 18, 15, 79, 56, 80]],
 [[81, 82, 83, 84, 85, 34, 86],
  [56, 87, 88, 89, 68, 34, 8, 90],
  [34, 91, 92, 93, 94, 95, 96, 52],
  [97, 76, 41, 98, 99, 100, 101, 102, 103],
  [104, 105, 106, 43, 107, 34, 108, 45],
  [43, 107, 15, 109, 100, 34, 110, 111],
  [28, 112, 61, 29, 30, 88, 113, 32],
  [114, 115, 116, 117, 56, 118, 119],
  [120, 121, 122, 119, 123, 34, 8, 124],
  [125, 26, 126, 127, 74, 128, 129, 100, 130],
  [84, 131, 132, 133, 56, 134, 132, 135, 136],
  [137, 21, 108, 18, 138, 29],
  [74, 114, 28, 76, 139, 140, 81, 26, 51, 135],
  [56, 141,

In [21]:
# Helpful lists
# Syllables
syllable_file = open("data/Syllable_dictionary.txt", 'r')
syllables = syllable_file.readlines()
syllables = [x.split() for x in syllables]
syllable_dict = {}

# We choose to map words to tuples of lists
# the first list corresponds to the number of syllables if the word were at the end (E)
# the second list corresponds to the number of syllables the word can take anywhere
# E.g. "test": ['E1', '2', '3'] <-> "test": [([1], [2, 3])]
for syllable in syllables:
    word = re.sub(r'[^\w]', '', syllable[0])
    end_syllable_list = []
    regular_syllable_list = []
    for item in syllable[1:]:
        if item[0] == "E":
            end_syllable_list.append(int(item[1:]))
        else:
            regular_syllable_list.append(int(item))
    syllable_dict[word] = (end_syllable_list, regular_syllable_list)
    
syllable_dict

{'gainst': ([], [1]),
 'greeing': ([1], [2]),
 'scaped': ([], [1]),
 'tis': ([], [1]),
 'twixt': ([], [1]),
 'a': ([], [1]),
 'adoting': ([2], [3]),
 'abhor': ([], [2]),
 'abide': ([], [2]),
 'able': ([], [2]),
 'about': ([], [2]),
 'above': ([], [2]),
 'absence': ([], [2]),
 'absent': ([], [2]),
 'abundance': ([], [3]),
 'abundant': ([], [3]),
 'abuse': ([], [2]),
 'abused': ([], [2]),
 'abuses': ([], [3]),
 'abysm': ([], [2]),
 'accents': ([], [2]),
 'acceptable': ([], [4]),
 'acceptance': ([], [3]),
 'accessary': ([], [4]),
 'accident': ([], [3]),
 'accidents': ([], [3]),
 'account': ([], [2]),
 'accumulate': ([], [4]),
 'accuse': ([], [2]),
 'accusing': ([], [3]),
 'achieve': ([], [2]),
 'acknowledge': ([], [3]),
 'acquaintance': ([], [3]),
 'acquainted': ([2], [3]),
 'act': ([], [1]),
 'action': ([], [2]),
 'active': ([], [2]),
 'actor': ([], [2]),
 'add': ([], [1]),
 'added': ([], [2]),
 'adders': ([], [2]),
 'addeth': ([], [2]),
 'adding': ([], [2]),
 'addition': ([], [3]),
 'ad

In [22]:
tokenized_syllable_dict = {}
for key in syllable_dict.keys():
    # If the word in syllable_dict is in our token map, add it to our tokenized_syllable_dict
    try:
        tokenized_syllable_dict[token_map[key]] = syllable_dict[key]
    except KeyError:
        pass
tokenized_syllable_dict

{476: ([], [1]),
 2594: ([1], [2]),
 2251: ([], [1]),
 925: ([], [1]),
 2016: ([], [1]),
 41: ([], [1]),
 841: ([2], [3]),
 3088: ([], [2]),
 1015: ([], [2]),
 2188: ([], [2]),
 2562: ([], [2]),
 2199: ([], [2]),
 1343: ([], [2]),
 1374: ([], [2]),
 44: ([], [3]),
 2359: ([], [3]),
 216: ([], [2]),
 2152: ([], [2]),
 2719: ([], [3]),
 2552: ([], [2]),
 1923: ([], [2]),
 234: ([], [4]),
 2905: ([], [3]),
 1272: ([], [4]),
 2754: ([], [3]),
 2605: ([], [3]),
 1120: ([], [2]),
 2648: ([], [4]),
 2638: ([], [2]),
 1721: ([], [3]),
 1888: ([], [2]),
 1293: ([], [3]),
 2066: ([], [3]),
 815: ([2], [3]),
 3100: ([], [1]),
 1842: ([], [2]),
 1303: ([], [2]),
 890: ([], [2]),
 1929: ([], [1]),
 2079: ([], [2]),
 2555: ([], [2]),
 2906: ([], [2]),
 844: ([], [2]),
 842: ([], [3]),
 1703: ([], [2]),
 2269: ([], [2]),
 2743: ([], [2]),
 2175: ([], [3]),
 1749: ([], [3]),
 2620: ([], [2]),
 2908: ([], [3]),
 1622: ([], [3]),
 358: ([], [2]),
 2713: ([], [3]),
 2087: ([], [2]),
 1824: ([], [3]),
 12

## Syllable Analysis - is in HMM_helper file (generate_emissions)

We deliberately do not make a 140 syllable line and then split them (which would allow for some extra continuity) because Shakespearean lines are typically treated as new sentences.

In [23]:
# Flattens 3-dimensional list of list of lists to 2-dimensional list of lists
# (where each inner list corresponds to a line of a poem)
flattened_tokenized_poems = [val for sublist in tokenized_poems for val in sublist]
# flattened_poems = [val for sublist in tokenized_poems for val in sublist]
hmm = unsupervised_HMM(flattened_tokenized_poems, 2, 10)
print('Rhyming Sonnet:\n====================')
for i in range(1):
    print(Meter_Rhyme_HMM_helper.sample_pair(hmm, token_map, tokenized_syllable_dict, num_syllables=10))

Iteration: 10
Rhyming Sonnet:


TypeError: sample_pair() missing 1 required positional argument: 'rhyming_dict'

## Rhyme Analysis

In [24]:
print(Meter_Rhyme_HMM_helper.sample_sentence(hmm, token_map, tokenized_syllable_dict, num_syllables=10))

AttributeError: 'HiddenMarkovModel' object has no attribute 'generate_emission'

## Meter Analysis
### Note: we call words that can have different number of syllables ambiguous_syllabic_words

In [25]:
# Contains word mapped to list of possible syllables (merges end and regular list too)
flattened_syllable_dict = {}
for key in syllable_dict.keys():
    flattened_syllable_dict[key] = [item for sublist in list(syllable_dict[key]) for item in sublist]

flattened_tokenized_syllable_dict = {}
for key in tokenized_syllable_dict.keys():
    flattened_tokenized_syllable_dict[key] = [item for sublist in list(tokenized_syllable_dict[key]) for item in sublist]

print ("Maximum number of possible syllables for any word: {}" \
       .format(max([len(x) for x in flattened_syllable_dict.values()])))
flattened_syllable_dict

Maximum number of possible syllables for any word: 2


{'gainst': [1],
 'greeing': [1, 2],
 'scaped': [1],
 'tis': [1],
 'twixt': [1],
 'a': [1],
 'adoting': [2, 3],
 'abhor': [2],
 'abide': [2],
 'able': [2],
 'about': [2],
 'above': [2],
 'absence': [2],
 'absent': [2],
 'abundance': [3],
 'abundant': [3],
 'abuse': [2],
 'abused': [2],
 'abuses': [3],
 'abysm': [2],
 'accents': [2],
 'acceptable': [4],
 'acceptance': [3],
 'accessary': [4],
 'accident': [3],
 'accidents': [3],
 'account': [2],
 'accumulate': [4],
 'accuse': [2],
 'accusing': [3],
 'achieve': [2],
 'acknowledge': [3],
 'acquaintance': [3],
 'acquainted': [2, 3],
 'act': [1],
 'action': [2],
 'active': [2],
 'actor': [2],
 'add': [1],
 'added': [2],
 'adders': [2],
 'addeth': [2],
 'adding': [2],
 'addition': [3],
 'adieu': [2],
 'adjunct': [2],
 'admire': [2],
 'admired': [3],
 'admiring': [3],
 'admit': [2],
 'admitted': [3],
 'adonis': [3],
 'adore': [2],
 'adulterate': [3],
 'advance': [2],
 'advantage': [3],
 'adverse': [2],
 'advised': [2],
 'advocate': [3],
 'afar'

In [26]:
# Contains words from syllable_dict that can have different numbers of syllables
ambiguous_syllabic_words = {}
for word in flattened_syllable_dict.keys():
    if len(d[word]) > 1:
        ambiguous_syllabic_words[word] = syllable_dict[word]
ambiguous_syllabic_words

NameError: name 'd' is not defined

In [27]:
ambiguous_syllabic_word_tokens = set()
for word in ambiguous_syllabic_words:
    ambiguous_syllabic_word_tokens.add(token_map[word])

unambiguous_lines = [] # list of lines that contain 1 or fewer words that can have many numbers of syllables
ambiguous_lines = [] # list of lines that contain 2 or more words that can have many numbers of syllables
for line in flattened_tokenized_poems:
    count = 0
    for token in line:
        if token in ambiguous_syllabic_word_tokens:
            count += 1
    if count <= 1:
        unambiguous_lines.append(line)
    else:
        ambiguous_lines.append(line)
print("Number of 'unambiguous lines': {}".format(len(unambiguous_lines)))
print("Number of 'ambiguous lines': {}".format(len(ambiguous_lines)))

Number of 'unambiguous lines': 2128
Number of 'ambiguous lines': 0


We now have a list of 'unambiguous' lines from the poems that have at most one ambiguous-syllabic word. It is easy to use this to figure out whether a word starts with a stressed or unstressed syllable in the 'unambiguous' lines. We manually identify all the remaining words in the 'ambiguous' lines.

In [28]:
# Note: Some Shakespeare lines do not have 10 syllables! We assume that they end in an unstressed syllable
# (the unstressed-stressed pattern continues)
weird = []
for line in unambiguous_lines:
    syllable_count_1 = 0 # must end at 10 (iambic pentameter) at the end of the line
    syllable_count_2 = 0 # must end at 10 (iambic pentameter) at the end of the line
    for word in line:
        syllable_count_1 += flattened_tokenized_syllable_dict[word][0]
        syllable_count_2 += flattened_tokenized_syllable_dict[word][-1]
    if syllable_count_1 != 10 and syllable_count_2 != 10:
        weird.append(line)
print("Lines that do not have 10 syllables:\n")
for line in weird:
    string = ""
    for word in line:
        string += token_map_r[word] + " "
    print (string)

Lines that do not have 10 syllables:

whose speechless song being many seeming one 
shifts but his place for still the world enjoys it 
and kept unused the user so destroys it 
bearing thy heart which i will keep so chary 
in thy souls thought all naked will bestow it 
then may i dare to boast how i do love thee 
till then not show my head where thou mayst prove me 
thou dost love her because thou knowst i love her 
and for my sake even so doth she abuse me 
suffring my friend for my sake to approve her 
being had to triumph being lacked to hope 
the spirit of love with a perpetual dulness 
being your vassal bound to stay your leisure 
whether we are mended or whether better they 
either not assailed or victor being charged 
my spirit is thine the better part of me 
knowing a better spirit doth use your name 
and their gross painting might be better used 
comes home again on better judgement making 
with mine own weakness being best acquainted 
all these i better in one general best 
t

In [29]:
# Words that start with a stressed syllable (initialize to all the words we had to manually categorize)
starts_stressed = set()
starts_unstressed = set() # words that start with an unstressed syllable
# We exploit that each unambiguous line has at most one ambiguous-syllabic word and that each ambiguous-syllabic
# word has at most 2 numbers of syllables.
# Note: Some Shakespeare lines do not have 10 syllables! We assume that they end in an unstressed syllable
# (the unstressed-stressed pattern continues)
for line in unambiguous_lines:
    # Choose the first number of syllables.
    # If syllable count is not 10 that way, choose the last number of syllables
    temp_starts_unstressed = set()
    temp_starts_stressed = set()
    
    syllable_count = 0 # generally ends at 10 (iambic pentameter) at the end of the line
    for word in line:
        if syllable_count %2 == 0:
            temp_starts_unstressed.add(word)
        else:
            temp_starts_stressed.add(word)
        syllable_count += flattened_tokenized_syllable_dict[word][0]
    if syllable_count == 10:
        starts_stressed = starts_stressed.union(temp_starts_stressed)
        starts_unstressed = starts_unstressed.union(temp_starts_unstressed)
        continue
    
    # Else syllable count != 10:
    syllable_count = 0
    for word in line:
        if syllable_count %2 == 0:
            starts_unstressed.add(word)
        else:
            starts_stressed.add(word)
        syllable_count += flattened_tokenized_syllable_dict[word][-1]

Finding the stressed/unstressed syllables per word is pretty difficult for the ambiguous lines so we manually classify the words we have not seen yet

In [30]:
unseen_words= set(tokenized_syllable_dict.keys()) - starts_stressed.union(starts_unstressed)
for word in unseen_words:
    print(token_map_r[word], word)
print (len(unseen_words))

0


In [31]:
starts_stressed = starts_stressed.union \
    (set([2575, 658, 1683, 1941, 1686, 1948, 1949, 2228, 2740, 1207, \
          2752, 2627, 1744, 1759, 1761, 2661, 881, 2296, 2301]))
starts_unstressed = starts_unstressed.union \
    (set([2574, 2722, 2762, 1760, 2302]))

In [32]:
print(set(tokenized_syllable_dict.keys()) - starts_stressed.union(starts_unstressed))
print(len(starts_stressed.intersection(starts_unstressed)))
print(len(starts_stressed.union(starts_unstressed)))
print(len(starts_stressed - starts_unstressed))
print(len(starts_unstressed - starts_stressed))

set()
495
3148
1854
799


In [33]:
# [token_map_r[x] for x in list(starts_stressed)]
# Let's see how well this analysis worked
# Words that only start stressed:
[token_map_r[x] for x in list(starts_stressed - starts_unstressed)]

['fairest',
 'creatures',
 'rose',
 'riper',
 'tender',
 'heir',
 'memory',
 'flame',
 'selfsubstantial',
 'fuel',
 'famine',
 'foe',
 'cruel',
 'ornament',
 'herald',
 'gaudy',
 'spring',
 'buriest',
 'niggarding',
 'world',
 'else',
 'glutton',
 'grave',
 'forty',
 'winters',
 'brow',
 'dig',
 'trenches',
 'field',
 'youths',
 'livery',
 'gazed',
 'tattered',
 'weed',
 'held',
 'asked',
 'treasure',
 'lusty',
 'sunken',
 'shame',
 'thriftless',
 'answer',
 'child',
 'sum',
 'blood',
 'feelst',
 'viewest',
 'mother',
 'uneared',
 'womb',
 'tillage',
 'husbandry',
 'fond',
 'tomb',
 'stop',
 'mothers',
 'back',
 'lovely',
 'april',
 'prime',
 'windows',
 'age',
 'wrinkles',
 'golden',
 'single',
 'image',
 'dies',
 'loveliness',
 'spend',
 'legacy',
 'lend',
 'frank',
 'lends',
 'free',
 'niggard',
 'bounteous',
 'largess',
 'given',
 'usurer',
 'sums',
 'traffic',
 'nature',
 'acceptable',
 'audit',
 'leave',
 'tombed',
 'used',
 'hours',
 'work',
 'frame',
 'gaze',
 'every',
 'dwell'

In [34]:
# Words that only start unstressed:
[token_map_r[x] for x in list(starts_unstressed - starts_stressed)]

['imprint',
 'desire',
 'increase',
 'eternity',
 'contain',
 'commit',
 'delivered',
 'acquaintance',
 'decease',
 'enrich',
 'invoked',
 'assistance',
 'contracted',
 'disperse',
 'aloft',
 'feedst',
 'lights',
 'compile',
 'advance',
 'decayed',
 'deserves',
 'abundance',
 'behaviour',
 'afford',
 'thereof',
 'spends',
 'within',
 'inferior',
 'broad',
 'content',
 'afloat',
 'tall',
 'forgotten',
 'immortal',
 'entombed',
 'oerread',
 'attaint',
 'besiege',
 'oerlook',
 'finding',
 'enforced',
 'anew',
 'timebettering',
 'devised',
 'plain',
 'truetelling',
 'gross',
 'abused',
 'exceed',
 'impute',
 'impair',
 'alleating',
 'confine',
 'immured',
 'example',
 'lean',
 'deserved',
 'couldst',
 'admired',
 'richly',
 'compiled',
 'unlettered',
 'proving',
 'succession',
 'amen',
 'affords',
 'warm',
 'refined',
 'hearing',
 'inhearse',
 'above',
 'compeers',
 'another',
 'giving',
 'repair',
 'astonished',
 'renewest',
 'familiar',
 'beguile',
 'unbless',
 'intelligence',
 'enfeeble

In [36]:
starts_unstressed

{0,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 24,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 37,
 40,
 41,
 43,
 44,
 45,
 46,
 48,
 49,
 51,
 52,
 53,
 54,
 56,
 57,
 61,
 62,
 64,
 65,
 66,
 67,
 68,
 70,
 72,
 74,
 76,
 77,
 78,
 80,
 81,
 84,
 85,
 88,
 92,
 94,
 96,
 97,
 100,
 101,
 102,
 104,
 105,
 107,
 108,
 111,
 112,
 114,
 115,
 116,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 130,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 164,
 165,
 168,
 171,
 172,
 175,
 177,
 179,
 183,
 185,
 188,
 189,
 192,
 193,
 197,
 199,
 201,
 203,
 204,
 205,
 206,
 207,
 211,
 212,
 214,
 216,
 220,
 221,
 223,
 225,
 226,
 227,
 229,
 230,
 232,
 233,
 237,
 238,
 240,
 242,
 243,
 244,
 246,
 248,
 252,
 257,
 258,
 260,
 262,
 266,
 267,
 268,
 269,
 270,
 272,
 273,
 274,
 278,
 283,
 284,
 285,
 286,
 287,

In [38]:
tokenized_syllable_dict[0]

([], [1])